<h1 align="center"> INF6804 Lab 1: Lucas Kanade Optical Flow </h1>
<br>
<h3 align="center"> Daniel Wang, Oumayma Messoussi </h3>
<br>

In [5]:
import cv2
import numpy as np
import glob
import argparse
import time, datetime
from matplotlib import pyplot as plt

#### Combining CDNet images into a video using OpenCV

In [2]:
# Source: https://theailearner.com/2018/10/15/creating-video-from-images-using-opencv-python/ (with modifications)
 
img_array = []
fps = 0

for folder in glob.glob('.\CDNet\*'):
    print('Video compilation for ' + folder)
    print('\tReading input images..')
    for filename in glob.glob(folder + '\*\input\*.jpg'):
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width, height)
        img_array.append(img)
    
    print('\tAll images read')
    print('\t\tCompiling ' + folder + '.mp4')
    
    if 'Low' in folder:
        fps = 1
    elif 'Baseline' in folder:
        fps = 30
    else:
        fps = 15
        
    print('\t\tFPS set to ' + str(fps))
    out = cv2.VideoWriter(folder + '-' + str(fps) + 'fps.mp4', cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
 
    for i in range(len(img_array)):
        out.write(img_array[i])
        
    print('\t\tVideo ' + folder + '-' + str(fps) + 'fps.mp4 ready\n')
    
out.release()

Video compilation for .\CDNet\Baseline
	Reading input images..
	All images read
		Compiling .\CDNet\Baseline.mp4
		FPS set to 30
		Video .\CDNet\Baseline-30fps.mp4 ready

Video compilation for .\CDNet\Low Framerate
	Reading input images..
	All images read
		Compiling .\CDNet\Low Framerate.mp4
		FPS set to 1
		Video .\CDNet\Low Framerate-1fps.mp4 ready

Video compilation for .\CDNet\Night Videos
	Reading input images..
	All images read
		Compiling .\CDNet\Night Videos.mp4
		FPS set to 15
		Video .\CDNet\Night Videos-15fps.mp4 ready



### OpenCV Lucas Kanade sparse optical flow test

In [3]:
# Source: https://docs.opencv.org/3.4/d4/dee/tutorial_optical_flow.html (with modifications)

# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 50,
                       qualityLevel = 0.2,
                       minDistance = 2,
                       blockSize = 7 )

# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15, 15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Create some random colors
color = np.random.randint(0, 255, (100, 3))

for video_path in glob.glob('.\CDNet\*.mp4'):

    print('Processing ' + video_path + ' ..')
    cap = cv2.VideoCapture(video_path)
    
    # Take first frame and find corners in it
    ret, old_frame = cap.read()
    
    kp_update = 0
    
    old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
    p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
    
    # Create a mask image for drawing purposes
    mask = np.zeros_like(old_frame)
    
    i = 0
    total_time = datetime.timedelta(seconds=0)
    
    while(1):
        ret, frame = cap.read()
        
        kp_update += 1
        if ret is True:
            frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        else:
            continue
        
        # calculate optical flow and run time
        prev_time = time.time()
        
        p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
        
        run_time = datetime.timedelta(seconds=time.time() - prev_time)
        total_time += run_time
        i += 1
        
        # Select good points
        good_new = p1[st==1]
        good_old = p0[st==1]
        
        # draw the tracks
        for i, (new, old) in enumerate(zip(good_new, good_old)):
            a, b = new.ravel()
            c, d = old.ravel()
            mask = cv2.line(mask, (a, b), (c, d), color[i].tolist(), 2)
            frame = cv2.circle(frame, (a, b), 5, color[i].tolist(), -1)
            
        img = cv2.add(frame, mask)
        cv2.imshow(video_path, img)
        
        k = cv2.waitKey(60) & 0xff
        if k == 27:
            break
        
        # Now update the previous frame and previous points
        old_gray = frame_gray.copy()
        p0 = good_new.reshape(-1, 1, 2)
        
        if kp_update == 10:
            p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
            kp_update = 0
        
    print('\tAverage run time for LK optical flow = ' + str(total_time/float(i)) + '\n')
    
cap.release()
cv2.destroyAllWindows()

NameError: name 'cap' is not defined

<br>

### Farneback dense optical flow test

In [2]:
# Source: https://docs.opencv.org/3.4/d4/dee/tutorial_optical_flow.html (with modifications)

for subset in glob.glob('.\CDNet\*'):

    print('Processing ' + subset + ' ..')
    
    i = 0
    
    for img in glob.glob(subset+'/*/input/*.jpg'):
    
        if i == 0:
            old_frame = cv2.imread(img)

            old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

            hsv = np.zeros_like(old_frame)
            hsv[...,1] = 255

            i += 1
            total_time = datetime.timedelta(seconds=0)

        else:
            i += 1
            
            frame = cv2.imread(img)

            frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # calculate optical flow and run time
            prev_time = time.time()

            flow = cv2.calcOpticalFlowFarneback(old_gray, frame_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)

            run_time = datetime.timedelta(seconds=time.time() - prev_time)
            total_time += run_time
            i += 1

            mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
            hsv[...,0] = ang*180/np.pi/2
            hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
            bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

            cv2.imshow('orig', frame)
            cv2.imshow(subset, bgr)

            k = cv2.waitKey(60) & 0xff
            if k == 27:
                break

            old_gray = frame_gray
        
    print('\tAverage run time for Farneback optical flow = ' + str(total_time/float(i)) + '\n')
    
cv2.destroyAllWindows()

Processing .\CDNet\Baseline ..
	Average run time for Farneback optical flow = 0:00:00.010573

Processing .\CDNet\Low Framerate ..
	Average run time for Farneback optical flow = 0:00:00.008391

Processing .\CDNet\Night Videos ..
	Average run time for Farneback optical flow = 0:00:00.037616

